## MODEL KANDİLLİ DATASET

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd

# Load the data from the text file
file_path = '/content/1915-2023.txt'  # Replace with your file path

# Try different encodings if 'utf-8' does not work
try:
    kandilli_data = pd.read_csv(file_path, encoding='utf-8',sep="\t")
except UnicodeDecodeError:
    kandilli_data = pd.read_csv(file_path, encoding='latin1',sep="\t")  # or 'iso-8859-1' or 'cp1252'

kandilli_data.head()


,No,Deprem Kodu,Olus tarihi,Olus zamani,Enlem,Boylam,Der(km),xM,MD,ML,Mw,Ms,Mb,Tip,Yer
0,1,20230831083800,2023.08.31,08:38:00.25,36.0547,27.1385,7.7,3.7,0.0,3.6,3.7,0.0,0.0,Ke,AKDENIZ
1,2,20230829060305,2023.08.29,06:03:05.24,38.0215,38.3377,6.3,3.9,0.0,3.8,3.9,0.0,0.0,Ke,YESILYAYLA-CELIKHAN (ADIYAMAN) [South 3.2 km]
2,3,20230829045814,2023.08.29,04:58:14.13,38.9088,32.9598,3.5,5.1,0.0,5.1,4.9,0.0,0.0,Ke,TAVSANCALI-KULU (KONYA) [South West 5.1 km]
3,4,20230827030942,2023.08.27,03:09:42.97,36.3487,27.8080,65.3,4.4,0.0,4.4,0.0,0.0,0.0,Ke,AKDENIZ
4,5,20230826091713,2023.08.26,09:17:13.88,35.8478,35.9930,8.1,3.6,0.0,3.5,3.6,0.0,0.0,Ke,YAYLADAGI (HATAY) [South West 8.8 km]


In [ ]:
kandilli_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20174 entries, 0 to 20173
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   No           20174 non-null  int64  
 1   Deprem Kodu  20174 non-null  int64  
 2   Olus tarihi  20174 non-null  object 
 3   Olus zamani  20174 non-null  object 
 4   Enlem        20174 non-null  float64
 5   Boylam       20174 non-null  float64
 6   Der(km)      20174 non-null  float64
 7   xM           20174 non-null  float64
 8   MD           20174 non-null  float64
 9   ML           20174 non-null  float64
 10  Mw           7193 non-null   float64
 11  Ms           20174 non-null  float64
 12  Mb           20174 non-null  float64
 13  Tip          20174 non-null  object 
 14  Yer          20174 non-null  object 
dtypes: float64(9), int64(2), object(4)
memory usage: 2.3+ MB


In [ ]:
# Gereksiz sütunları çıkar
kandilli_data.drop(['Deprem Kodu','Tip'], axis=1, inplace=True)

In [ ]:
# Tarih ve saat sütunlarını işleme
kandilli_data['Year'] = pd.to_datetime(kandilli_data['Olus tarihi']).dt.year
kandilli_data['Month'] = pd.to_datetime(kandilli_data['Olus tarihi']).dt.month
kandilli_data['Day'] = pd.to_datetime(kandilli_data['Olus tarihi']).dt.day
kandilli_data['Hour'] = pd.to_datetime(kandilli_data['Olus zamani']).dt.hour
kandilli_data['Minute'] = pd.to_datetime(kandilli_data['Olus zamani']).dt.minute
kandilli_data.drop(['Olus tarihi', 'Olus zamani'], axis=1, inplace=True)

In [ ]:
# Kategorik sütunları one-hot encoding ile dönüştürme
kandilli_data = pd.get_dummies(kandilli_data, columns=['Yer'])


In [ ]:
kandilli_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20174 entries, 0 to 20173
Columns: 12479 entries, No     to Yer_ÜVECIK AÇIKLARI-ÇANAKKALE (EGE DENIZI)
dtypes: float64(9), int64(6), uint8(12464)
memory usage: 242.1 MB


In [ ]:
 #sonn
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error
import numpy as np
 # Gereksiz sütunları çıkar
kandilli_data.drop(['xM', 'MD', 'ML', 'Ms', 'Mb'], axis=1, inplace=True)
# Handle missing values in 'Mw' column by replacing them with the mean
kandilli_data['Mw'].fillna(kandilli_data['Mw'].mean(), inplace=True)


# Veri setini özellikler ve hedef değişken olarak bölme
features = kandilli_data.drop('Mw', axis=1)
target = kandilli_data['Mw']

# Veri setini eğitim ve test setlerine bölme
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

# Özellikleri ölçeklendirme
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Yapay Sinir Ağı modelini oluşturma ve eğitme
model = MLPRegressor(hidden_layer_sizes=(100,), activation='relu', random_state=42, max_iter=500)
model.fit(X_train_scaled, y_train)

# Test verileri üzerinde tahmin yapma ve modelin performansını değerlendirme
predictions = model.predict(X_test_scaled)
rmse = np.sqrt(mean_squared_error(y_test, predictions))

rmse

2.034477801739453

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

# Veri setinizi yükleyin ve ön işlemleri uygulayın...


# Veri setini eğitim ve test setlerine bölme
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

# Özellikleri ölçeklendirme
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Hiperparametre ayarlamaları için GridSearchCV kullanımı
param_grid = {
    'hidden_layer_sizes': [(100,), (50, 50), (100, 50)],
    'activation': ['relu', 'tanh'],
    'learning_rate_init': [0.001, 0.01]
}

mlp = MLPRegressor(max_iter=1000, random_state=42)
grid_search = GridSearchCV(mlp, param_grid, cv=3, scoring='neg_mean_squared_error')
grid_search.fit(X_train_scaled, y_train)

# En iyi modeli bulma
best_model = grid_search.best_estimator_

# Test verileri üzerinde tahmin yapma ve modelin performansını değerlendirme
predictions = best_model.predict(X_test_scaled)
rmse = np.sqrt(mean_squared_error(y_test, predictions))
r2 = r2_score(y_test, predictions)

print("RMSE: ", rmse)
print("R^2: ", r2)


RMSE:  0.7898269339663255
R^2:  0.09572010312188739


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

# Veri setinizi yükleyin ve ön işlemleri uygulayın...


# Veri setini eğitim ve test setlerine bölme
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

# Özellikleri ölçeklendirme
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Rastgele Orman regresörü için parametre aralıklarını belirleme
param_distributions = {
    'n_estimators': [100, 200, 300, 400, 500],
    'max_depth': [None, 10, 20, 30, 40, 50],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# RandomizedSearchCV kullanarak modeli eğitme
rf = RandomForestRegressor(random_state=42)
random_search = RandomizedSearchCV(rf, param_distributions, n_iter=100, cv=3, verbose=2, random_state=42, n_jobs=-1)
random_search.fit(X_train_scaled, y_train)

# En iyi modeli bulma
best_rf_model = random_search.best_estimator_

# Test verileri üzerinde tahmin yapma ve modelin performansını değerlendirme
predictions = best_rf_model.predict(X_test_scaled)
rmse = np.sqrt(mean_squared_error(y_test, predictions))
r2 = r2_score(y_test, predictions)

print("RMSE: ", rmse)
print("R^2: ", r2)


Fitting 3 folds for each of 100 candidates, totalling 300 fits


KeyboardInterrupt: ignored

In [ ]:
# Model için girdi ve hedef sütunları ayırma
X = kandilli_data.drop('xM', axis=1)
y = kandilli_data['xM']

# Verileri ölçeklendirme
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Model oluşturma fonksiyonu
def create_model():
    model = Sequential()
    model.add(Dense(64, activation='relu', input_dim=X_scaled.shape[1]))
    model.add(Dropout(0.2))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

# K-Fold Cross Validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)
mse_scores = []
r2_scores = []

for train_index, test_index in kf.split(X_scaled):
    X_train, X_test = X_scaled[train_index], X_scaled[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    model = create_model()
    model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=0)

    predictions = model.predict(X_test)
    mse = mean_squared_error(y_test, predictions)
    r2 = r2_score(y_test, predictions)

    mse_scores.append(mse)
    r2_scores.append(r2)

# Sonuçların ortalamasını hesaplama
avg_mse = np.mean(mse_scores)
avg_r2 = np.mean(r2_scores)

print("Ortalama MSE:", avg_mse)
print("Ortalama R^2 Score:", avg_r2)

127/127 [==============================] - 0s 2ms/step
Ortalama MSE: 0.6661339827035793
Ortalama R^2 Score: -1.3244020482562764
